# Silver Data Exploration

This notebook explores the **silver layer** data - the final processed output.

**Silver Layer Characteristics:**
- Schema-enforced data (strict column definitions)
- UUIDs generated for all entities
- Data linked via foreign keys (vel_id)
- Derived fields created (addresses, department codes)
- Business logic applied
- Production-ready datasets

In [1]:
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', None)

# Define paths
SILVER_PATH = Path('../data/silver/2024')
print(f"Silver data path: {SILVER_PATH}")
print(f"Files: {list(SILVER_PATH.glob('*.csv'))}")

Silver data path: ../data/silver/2024
Files: [PosixPath('../data/silver/2024/health_metrics.csv'), PosixPath('../data/silver/2024/qualifications.csv'), PosixPath('../data/silver/2024/etablissements.csv')]


## 1. Etablissements (Establishments)

Main entity table with all healthcare establishments

In [2]:
# Load establishments
df_etablissements = pd.read_csv(SILVER_PATH / 'etablissements.csv')

print(f"Shape: {df_etablissements.shape}")
print(f"\nColumns ({len(df_etablissements.columns)}):")
for col in df_etablissements.columns:
    print(f"  - {col}")

print("\nSample:")
df_etablissements.head(3)

Shape: (204915, 13)

Columns (13):
  - vel_id
  - finess_et
  - siret
  - raison_sociale
  - categorie_etab
  - categorie_detail
  - adresse_postale
  - code_postal
  - departement
  - date_created
  - date_updated
  - source
  - freshness

Sample:


,vel_id,finess_et,siret,raison_sociale,categorie_etab,categorie_detail,adresse_postale,code_postal,departement,date_created,date_updated,source,freshness
0,6fd5fd0f-e5eb-4fb6-b6d7-2d312d005b43,010000032,2.601000e+13,CENTRE HOSPITALIER BUGEY SUD,Public,Etablissement public de santé,700.0 AV DE NARVIK BP 139,1300.0,1.0,2026-01-21 17:11:07.429055,2026-01-21 17:11:07.429858,Data.gouv,NaN
1,128778e2-3111-4dea-85b6-166aeeba1c25,010000065,2.601003e+13,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,Public,Etablissement public de santé,14.0 R DE L'HOPITAL BP 615,1606.0,1.0,2026-01-21 17:11:07.429055,2026-01-21 17:11:07.429858,Data.gouv,NaN
2,b8d2070f-174b-4447-a508-cf9a20d361e8,010000081,2.601001e+13,CENTRE HOSPITALIER DU PAYS DE GEX,Public,Etablissement public de santé,160.0 R MARC PANISSOD BP 437,1174.0,1.0,2026-01-21 17:11:07.429055,2026-01-21 17:11:07.429858,Data.gouv,NaN


In [3]:
# Key statistics
print("Etablissements Statistics:")
print(f"  - Total establishments: {len(df_etablissements):,}")
print(f"  - Unique FINESS IDs: {df_etablissements['finess_et'].nunique():,}")
print(f"  - Unique vel_id (UUIDs): {df_etablissements['vel_id'].nunique():,}")
print(f"\nBy Category:")
print(df_etablissements['categorie_etab'].value_counts())
print(f"\nBy Department (top 10):")
print(df_etablissements['departement'].value_counts().head(10))

Etablissements Statistics:
  - Total establishments: 204,915
  - Unique FINESS IDs: 176,301
  - Unique vel_id (UUIDs): 204,915

By Category:
categorie_etab
Autre     196670
Public      6274
Privé       1971
Name: count, dtype: int64

By Department (top 10):
departement
59.0    4031
75.0    3525
13.0    3068
69.0    2886
92.0    2199
33.0    2173
62.0    2070
93.0    2038
76.0    1988
44.0    1962
Name: count, dtype: int64


In [4]:
# Data quality
print("Data Quality:")
print(f"  - Missing vel_id: {df_etablissements['vel_id'].isna().sum()}")
print(f"  - Missing finess_et: {df_etablissements['finess_et'].isna().sum()}")
print(f"  - Missing raison_sociale: {df_etablissements['raison_sociale'].isna().sum()}")
print(f"  - Missing code_postal: {df_etablissements['code_postal'].isna().sum()}")

Data Quality:
  - Missing vel_id: 0
  - Missing finess_et: 0
  - Missing raison_sociale: 0
  - Missing code_postal: 102458


## 2. Qualifications (HAS Certifications)

HAS certification data linked to establishments

In [ ]:
# Load qualifications
df_qualifications = pd.read_csv(SILVER_PATH / 'qualifications.csv')

print(f"Shape: {df_qualifications.shape}")
print(f"\nColumns ({len(df_qualifications.columns)}):")
for col in df_qualifications.columns:
    print(f"  - {col}")

print("\nSample:")
df_qualifications.head(10)

Shape: (15767, 9)

Columns (9):
  - qua_id
  - vel_id
  - niveau_certification
  - date_visite
  - url_rapport
  - date_created
  - date_updated
  - source
  - freshness

Sample:


,qua_id,vel_id,niveau_certification,date_visite,url_rapport,date_created,date_updated,source,freshness
0,d0c9a852-e384-456a-8183-8f2593bdd466,2d67a853-f0f5-49c7-94ac-724b06f78b3f,Certifié,2022-02-10,https://www.has-sante.fr/jcms/c_30001,2026-01-21 17:11:07.494073,2026-01-21 17:11:07.494298,HAS,Bisannuelle
1,d0c9a852-e384-456a-8183-8f2593bdd466,5df34073-ef47-4801-b458-93fc1520e321,Certifié,2022-02-10,https://www.has-sante.fr/jcms/c_30001,2026-01-21 17:11:07.494073,2026-01-21 17:11:07.494298,HAS,Bisannuelle
2,bc9c51e3-06b1-41b5-97fc-48da179c9b0f,a593691d-77cd-4b77-ab2e-68dc82c3c634,Certifié avec mention,2023-03-08,https://www.has-sante.fr/jcms/c_30002,2026-01-21 17:11:07.494073,2026-01-21 17:11:07.494298,HAS,Bisannuelle


In [6]:
# Certification statistics
print("Qualification Statistics:")
print(f"  - Total records: {len(df_qualifications):,}")
print(f"  - Unique establishments: {df_qualifications['vel_id'].nunique():,}")
print(f"  - Unique certification IDs: {df_qualifications['qua_id'].nunique():,}")
print(f"\nBy Certification Level:")
print(df_qualifications['niveau_certification'].value_counts())

Qualification Statistics:
  - Total records: 15,767
  - Unique establishments: 15,767
  - Unique certification IDs: 7,884

By Certification Level:
niveau_certification
Certifié                    10103
Certifié avec mention        2978
Certifié sous conditions     1148
Non certifié                  792
Name: count, dtype: int64


In [7]:
# Check linkage
print("Linkage Quality:")
print(f"  - Qualifications with missing vel_id: {df_qualifications['vel_id'].isna().sum()}")
print(f"  - vel_id values that exist in etablissements: ", end="")

# Check how many vel_id in qualifications exist in etablissements
valid_links = df_qualifications['vel_id'].isin(df_etablissements['vel_id']).sum()
print(f"{valid_links:,} / {len(df_qualifications):,} ({valid_links/len(df_qualifications)*100:.1f}%)")

Linkage Quality:
  - Qualifications with missing vel_id: 0
  - vel_id values that exist in etablissements: 15,767 / 15,767 (100.0%)


## 3. Health Metrics (IQSS)

Quality scores linked to establishments

In [ ]:
# Load health metrics
df_health_metrics = pd.read_csv(SILVER_PATH / 'health_metrics.csv')

print(f"Shape: {df_health_metrics.shape}")
print(f"\nColumns ({len(df_health_metrics.columns)}):")
for col in df_health_metrics.columns:
    print(f"  - {col}")

print("\nSample:")
df_health_metrics.head(10)

Shape: (1384, 16)

Columns (16):
  - metric_id
  - vel_id
  - score_all_ssr_ajust
  - score_ajust_esatis_region
  - score_accueil_ssr_ajust
  - score_pec_ssr_ajust
  - score_lieu_ssr_ajust
  - score_repas_ssr_ajust
  - score_sortie_ssr_ajust
  - classement
  - evolution
  - participation
  - depot
  - annee
  - date_created
  - source

Sample:


,metric_id,vel_id,score_all_ssr_ajust,score_ajust_esatis_region,score_accueil_ssr_ajust,score_pec_ssr_ajust,score_lieu_ssr_ajust,score_repas_ssr_ajust,score_sortie_ssr_ajust,classement,evolution,participation,depot,annee,date_created,source
0,bc7fff44-ec49-403e-a08b-531b18509711,f322fc95-6d91-4c41-9842-bfaeb1f56402,72.74,76.82,80.87,77.30,75.44,53.79,63.91,C,Stable,Facultatif,Oui,2024,NaN,IQSS
1,bc7fff44-ec49-403e-a08b-531b18509711,70930eea-94d8-4662-a2f5-111efa8d6627,72.74,76.82,80.87,77.30,75.44,53.79,63.91,C,Stable,Facultatif,Oui,2024,NaN,IQSS
2,f216dd81-9d24-4ee8-b68b-ec2e7abef408,42fe414f-2547-4cf6-8d84-fb34a35a4bbe,75.12,76.82,83.48,79.72,75.86,54.45,70.06,B,Stable,Facultatif,Oui,2024,NaN,IQSS


In [9]:
# Metrics statistics
print("Health Metrics Statistics:")
print(f"  - Total records: {len(df_health_metrics):,}")
print(f"  - Unique establishments: {df_health_metrics['vel_id'].nunique():,}")
print(f"  - Unique metric IDs: {df_health_metrics['metric_id'].nunique():,}")

# Score columns
score_cols = [col for col in df_health_metrics.columns if 'score' in col]
print(f"\nScore columns ({len(score_cols)}):")
for col in score_cols:
    print(f"  - {col}")

Health Metrics Statistics:
  - Total records: 1,384
  - Unique establishments: 1,382
  - Unique metric IDs: 692

Score columns (7):
  - score_all_ssr_ajust
  - score_ajust_esatis_region
  - score_accueil_ssr_ajust
  - score_pec_ssr_ajust
  - score_lieu_ssr_ajust
  - score_repas_ssr_ajust
  - score_sortie_ssr_ajust


In [10]:
# Score distributions
print("Score Distributions:")
for col in score_cols:
    if df_health_metrics[col].notna().sum() > 0:
        print(f"\n{col}:")
        print(df_health_metrics[col].describe())

Score Distributions:

score_all_ssr_ajust:
count    788.000000
mean      75.751574
std        4.604680
min       50.750000
25%       73.010000
50%       76.150000
75%       79.190000
max       86.520000
Name: score_all_ssr_ajust, dtype: float64

score_ajust_esatis_region:
count    788.000000
mean      76.002437
std        1.585459
min       71.370000
25%       75.360000
50%       76.050000
75%       76.820000
max       78.910000
Name: score_ajust_esatis_region, dtype: float64

score_accueil_ssr_ajust:
count    788.000000
mean      82.335584
std        5.473874
min       50.900000
25%       79.080000
50%       83.000000
75%       86.440000
max       94.660000
Name: score_accueil_ssr_ajust, dtype: float64

score_pec_ssr_ajust:
count    788.000000
mean      79.122284
std        4.958726
min       48.410000
25%       76.040000
50%       79.970000
75%       82.680000
max       87.640000
Name: score_pec_ssr_ajust, dtype: float64

score_lieu_ssr_ajust:
count    788.000000
mean      77.193020


In [11]:
# Classifications
if 'classement' in df_health_metrics.columns:
    print("Classification Distribution:")
    print(df_health_metrics['classement'].value_counts())

if 'evolution' in df_health_metrics.columns:
    print("\nEvolution Distribution:")
    print(df_health_metrics['evolution'].value_counts())

Classification Distribution:
classement
DI    556
A     362
B     234
C     142
D      50
NV     36
NR      4
Name: count, dtype: int64

Evolution Distribution:
evolution
Stable            378
Amélioration      192
Non calculable    162
Diminution         56
Name: count, dtype: int64


## 4. Data Relationships

Verify the foreign key relationships between tables

In [ ]:
# Join example: Establishments with Qualifications
df_joined = pd.merge(
    df_etablissements[['vel_id', 'finess_et', 'raison_sociale', 'categorie_etab', 'departement']],
    df_qualifications[['vel_id', 'niveau_certification', 'date_visite']],
    on='vel_id',
    how='inner'
)

print(f"Joined Establishments + Qualifications: {len(df_joined):,} records")
print("\nSample of joined data:")
df_joined.head(10)

Joined Establishments + Qualifications: 15,767 records

Sample of joined data:


,vel_id,finess_et,raison_sociale,categorie_etab,departement,niveau_certification,date_visite
0,6fd5fd0f-e5eb-4fb6-b6d7-2d312d005b43,010000032,CENTRE HOSPITALIER BUGEY SUD,Public,1.0,Certifié,2025-05-14
1,128778e2-3111-4dea-85b6-166aeeba1c25,010000065,CENTRE HOSPITALIER DE TREVOUX - MONTPENSIER,Public,1.0,Certifié,2022-09-27
2,b8d2070f-174b-4447-a508-cf9a20d361e8,010000081,CENTRE HOSPITALIER DU PAYS DE GEX,Public,1.0,Certifié,2024-03-06
3,9c469967-bf4a-4374-97e1-ed00faf3a109,010000099,CENTRE HOSPITALIER DE MEXIMIEUX,Public,1.0,Certifié,2024-03-20
4,074b42d7-db0d-4b42-b147-bd7aa5cae6dc,010000107,CENTRE HOSPITALIER DE PONT DE VAUX,Public,1.0,Certifié sous conditions,2025-01-15


In [13]:
# Join example: Establishments with Health Metrics
df_joined_metrics = pd.merge(
    df_etablissements[['vel_id', 'finess_et', 'raison_sociale', 'departement']],
    df_health_metrics[['vel_id', 'score_all_ssr_ajust', 'classement']],
    on='vel_id',
    how='inner'
)

print(f"Joined Establishments + Health Metrics: {len(df_joined_metrics):,} records")
print("\nSample of joined data:")
df_joined_metrics.head()

Joined Establishments + Health Metrics: 1,384 records

Sample of joined data:


,vel_id,finess_et,raison_sociale,departement,score_all_ssr_ajust,classement
0,f322fc95-6d91-4c41-9842-bfaeb1f56402,010002129,CLINIQUE LES ARBELLES,1.0,72.74,C
1,42fe414f-2547-4cf6-8d84-fb34a35a4bbe,010008852,SMR ORSAC DE L'AIN - SITE DE MARIE GAVOTY,1.0,75.12,B
2,a5f473a4-ae55-455f-9527-8480beaf3139,010011641,CLINIQUE DU SOUFFLE LE PONTET,1.0,79.06,A
3,102ca913-6eb3-42f1-86df-02b4b7600b6e,010780278,SMR ORSAC DE L AIN SITE DE FELIX MANGINI,1.0,76.97,A
4,c50630de-6eee-49f8-aa40-d4336754718d,010780476,CENTRE SOINS DE SUITE ET READAPT POUR ADOLESCE...,1.0,NaN,DI


## Summary

**Silver Layer Statistics:**
- Total establishments: {len(df_etablissements):,}
- Linked qualifications: {len(df_qualifications):,}
- Linked health metrics: {len(df_health_metrics):,}

**Data Quality:**
- ✓ All entities have UUIDs
- ✓ Foreign key relationships intact
- ✓ Schema-enforced columns
- ✓ Business logic applied
- ✓ Ready for analytics and applications

**Pipeline Success:**
```
Raw (mixed formats) 
  → Cleaning → Bronze (standardized CSV)
  → Processing → Silver (schema-enforced, linked)
```